# L09 16/04/24

# CNN, Lucas database

In [1]:
import re
import pandas as pd
import torchmetrics
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import datetime as dt
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.preprocessing import OneHotEncoder
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm


/home/omirako/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-30 15:55:50.581542: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 15:55:51.249223: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../datasets/Lucas/lucas_dataset_train.csv', sep=',').drop(columns=['Unnamed: 0'])
df

,spc.400,spc.400.5,spc.401,spc.401.5,spc.402,spc.402.5,spc.403,spc.403.5,spc.404,spc.404.5,...,sand,pH.in.CaCl2,pH.in.H2O,OC,CaCO3,N,P,K,CEC,set
0,0.669077,0.676745,0.684369,0.691928,0.699396,0.706758,0.713992,0.721079,0.728002,0.734743,...,75,4.78,5.15,8.2,0,0.9,39.6,54.6,2.8,2
1,0.679681,0.687950,0.696161,0.704279,0.712283,0.720136,0.727818,0.735309,0.742584,0.749629,...,41,4.33,4.84,44.1,0,2.5,54.2,261.8,13.8,1
2,0.786848,0.795459,0.804018,0.812496,0.820865,0.829104,0.837181,0.845079,0.852777,0.860258,...,48,6.08,6.67,22.6,0,2.3,29.1,216.5,12.8,1
3,0.583825,0.592186,0.600491,0.608715,0.616834,0.624822,0.632655,0.640310,0.647767,0.655004,...,35,4.67,5.58,21.0,0,1.5,0.0,69.8,4.7,1
4,0.791126,0.799194,0.807208,0.815131,0.822943,0.830612,0.838116,0.845435,0.852549,0.859437,...,50,6.77,7.04,38.8,6,3.0,12.1,54.3,20.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,0.642357,0.650594,0.658771,0.666853,0.674813,0.682620,0.690251,0.697683,0.704898,0.711878,...,86,3.72,4.66,10.3,0,0.3,15.5,21.0,3.0,3
13935,0.922261,0.934263,0.946202,0.958037,0.969738,0.981275,0.992614,1.003729,1.014598,1.025192,...,22,7.01,7.69,11.3,2,1.0,0.0,458.5,22.6,3
13936,0.705685,0.712331,0.718935,0.725476,0.731939,0.738299,0.744541,0.750645,0.756600,0.762389,...,89,3.09,3.95,9.3,0,0.4,0.0,20.3,4.7,1
13937,0.821155,0.831790,0.842372,0.852866,0.863251,0.873497,0.883579,0.893470,0.903146,0.912586,...,53,7.02,7.68,6.2,1,0.7,0.0,466.0,17.1,1


In [3]:

import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define the dataloader
class Dataset(torch.utils.data.Dataset):
    def __init__(self, df, target):
        self.data = torch.tensor(df, dtype=torch.float32).unsqueeze(1) # Add a channel dimension
        self.target = torch.tensor(target.values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

# define the net
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=32, kernel_size=3, stride=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.conv2 = nn.Sequential( 
            nn.Conv1d(in_channels=32, out_channels=128, kernel_size=3, stride=2),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=2),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16640, 4096),
            nn.ReLU(),
            nn.Dropout(0.2)
        )

        self.fc4 = nn.Sequential(
            nn.Linear(4096, 12),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc4(x)
        
        return x


In [4]:
class CNet(nn.Module):
    def __init__(self):
        super(CNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=11, stride=4, padding=1),
            nn.BatchNorm1d(device=device, num_features=64),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=64, out_channels=64, kernel_size=7, stride=2, padding=1),
            nn.BatchNorm1d(device=device, num_features=64),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=3, stride=3)
        )

        self.conv2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(device=device, num_features=128),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(device=device, num_features=128),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )
        
        self.conv3 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
            nn.BatchNorm1d(device=device, num_features=256),
            nn.LeakyReLU(),
            nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, stride=1),
            nn.BatchNorm1d(device=device, num_features=256),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(10496, 2048),
            nn.BatchNorm1d(device=device, num_features=2048),
            nn.LeakyReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(2048, 2048),
            nn.BatchNorm1d(device=device, num_features=2048),
            nn.LeakyReLU()
        )

        self.fc3 = nn.Linear(2048, 12)

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [5]:
net = CNet()
net.to(device)
inp = torch.rand(100, 1, 4200).to(device)
out = net(inp)
print(out.shape)

del net, inp, out

torch.Size([100, 12])


In [6]:
df[df.columns[~df.columns.str.contains('spc')]]

,GPS_LAT,GPS_LONG,coarse,clay,silt,sand,pH.in.CaCl2,pH.in.H2O,OC,CaCO3,N,P,K,CEC,set
0,51.88810,22.902990,9,3,22,75,4.78,5.15,8.2,0,0.9,39.6,54.6,2.8,2
1,66.79031,22.910723,25,12,48,41,4.33,4.84,44.1,0,2.5,54.2,261.8,13.8,1
2,52.78472,-2.174444,8,18,34,48,6.08,6.67,22.6,0,2.3,29.1,216.5,12.8,1
3,43.88222,2.050493,18,17,48,35,4.67,5.58,21.0,0,1.5,0.0,69.8,4.7,1
4,55.85009,23.185540,8,12,38,50,6.77,7.04,38.8,6,3.0,12.1,54.3,20.5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13934,58.97335,13.211217,9,3,11,86,3.72,4.66,10.3,0,0.3,15.5,21.0,3.0,3
13935,36.89048,22.713300,14,41,37,22,7.01,7.69,11.3,2,1.0,0.0,458.5,22.6,3
13936,50.98141,17.548190,9,3,8,89,3.09,3.95,9.3,0,0.4,0.0,20.3,4.7,1
13937,37.07633,14.520210,1,34,13,53,7.02,7.68,6.2,1,0.7,0.0,466.0,17.1,1


In [7]:
# Define the train
df_val = pd.read_csv('../datasets/Lucas/lucas_dataset_val.csv', sep=',').drop(columns=['Unnamed: 0'])


X_train = df[df.columns[df.columns.str.contains('spc') | df.columns.str.contains('GPS')]]
y_train = df[df.columns[~(df.columns.str.contains('spc') | df.columns.str.contains('GPS'))]].drop(columns=['set'])

X_val = df_val[df_val.columns[df_val.columns.str.contains('spc') | df_val.columns.str.contains('GPS')]]
y_val = df_val[df_val.columns[~(df_val.columns.str.contains('spc') | df_val.columns.str.contains('GPS'))]].drop(columns=['set'])

# Normalize the data
scaler = sklearn.preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)



train_dataset = Dataset(X_train, y_train)
val_dataset = Dataset(X_val, y_val)

In [8]:
y_val

,coarse,clay,silt,sand,pH.in.CaCl2,pH.in.H2O,OC,CaCO3,N,P,K,CEC
0,19,14,55,32,4.94,5.62,14.7,0,1.6,77.4,188.9,6.3
1,5,50,47,3,6.47,6.75,66.2,7,4.5,56.1,315.2,39.4
2,3,27,15,58,6.85,7.27,14.1,3,1.6,24.4,374.8,13.0
3,6,7,22,70,4.98,5.47,16.2,2,1.1,25.1,81.2,5.8
4,56,37,41,22,7.37,7.91,11.5,530,1.1,0.0,159.1,25.5
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,11,46,41,12,4.78,5.46,71.4,2,6.2,0.0,207.9,24.5
1996,12,14,37,48,3.84,4.32,76.6,6,4.4,0.0,59.8,8.6
1997,26,3,17,81,3.68,4.67,24.0,0,1.1,16.2,50.6,4.8
1998,7,6,35,59,4.84,5.33,14.7,0,1.4,47.4,78.4,6.4


In [9]:
def validate(model, val_loader):
    val_loss = 0.0
    num_val_samples = 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = F.mse_loss(output.squeeze(), target)
            val_loss += loss.item() * data.size(0)
            num_val_samples += data.size(0)
    return val_loss / num_val_samples


def train(model,  train_loader, val_loader, writer, optimizer, reload='No', custom = None ,epochs=2500):
    
    try:
        pbar.close()
    except:
        pass
    
    try:
        best_m = model.__class__().to(device)
        best_m.load_state_dict(torch.load('../datasets/Lucas/best_model.pth'))
        best_loss = validate(best_m, val_loader)
        print(f'Best model found with loss: {best_loss}')
    except Exception as e:
        best_loss = np.inf
        print('No best model found')
        print(e)
    
    if reload == 'last':
        try:
            model.load_state_dict(torch.load('../datasets/Lucas/last_model.pth'))
            print('Last Model loaded')
        except:
            print('Tried to load last model, not found')

    elif reload == 'best':
        model.load_state_dict(torch.load('../datasets/Lucas/best_model.pth'))
        print('Best Model loaded')
    
        
    
    n_steps = 0
    pbar = tqdm(total=epochs, desc="Training", leave=True)
    best_running_loss = np.inf
    best_running_epoch = 0
    
    for epoch in range(epochs):
        
        # Set the model to train mode and add the epoch number to tensorboard
        model.train()
        writer.add_scalar("epoch", epoch, n_steps)
        
        
        # Train the model
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.mse_loss(output.squeeze(), target)
            writer.add_scalar("loss", loss.item(), n_steps)  # Add loss to tensorboard
            loss.backward()
            optimizer.step()
            n_steps += 1

        # Validate the model
        val_loss = validate(model, val_loader)
        pbar.set_description(f"Training - Validation Loss: {val_loss:.4f} || Best running loss: {best_running_loss:.4f}, found at epoch {best_running_epoch} ({epoch-best_running_epoch} ago)")
        writer.add_scalar("val_loss", val_loss, n_steps)  # Add validation loss to tensorboard
        torch.save(model.state_dict(), '../datasets/Lucas/last_model.pth')
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), '../datasets/Lucas/best_model.pth')
        if val_loss < best_running_loss:
            best_running_loss = val_loss
            best_running_epoch = epoch
        pbar.update(1)  # Update tqdm progress bar
        
    pbar.close()
    
    
# set description tqdm

In [10]:
import gc

model = CNet().to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
batch_size = 2048
epochs = 2000
num_workers = 4
n_iter = 0

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

writer = SummaryWriter('Lucas_experiment')

train(model, reload='last', epochs=epochs, train_loader=train_loader, val_loader=val_loader, writer=writer, optimizer=optimizer)

Best model found with loss: 6477.83056640625
Last Model loaded


Training - Validation Loss: 2363.9893 || Best running loss: 2332.3206, found at epoch 1988 (11 ago): 100%|██████████| 2000/2000 [41:11<00:00,  1.24s/it] 


In [11]:
del model
del train_loader
del val_loader
gc.collect()
torch.cuda.empty_cache()